# Training the U-net model to segment mammogram images

In [1]:
# Import necessary libraries for deep learning model building
from keras.models import Model
from keras.layers import Layer, Conv2D, Dropout, UpSampling2D, concatenate, Add, Multiply, Input, MaxPool2D, BatchNormalization
from keras import backend as K
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from keras.metrics import MeanIoU

# Import necessary libraries for logic, tables, open cv and pydicom for data manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
import glob
import pydicom
import random
from pathlib import Path
from sklearn.model_selection import train_test_split
import shutil
import albumentations as A
import plistlib
from skimage.draw import polygon
import re
from tqdm import tqdm_notebook, tnrange
from glob import glob
from itertools import chain
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize

# from skimage.morphology import label  # You've commented this line, so I'm leaving it out
from tensorflow import keras
from skimage.color import rgb2gray
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import plot_model

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Make sure deep learning specific libraries are imported
from keras.models import Model
from keras.layers import Layer
from keras.layers import Conv2D
from keras.layers import Dropout
from keras.layers import UpSampling2D
from keras.layers import concatenate
from keras.layers import Add
from keras.layers import Multiply
from keras.layers import Input
from keras.layers import MaxPool2D
from keras.layers import BatchNormalization
from keras import backend as K 
from keras.callbacks import Callback
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

# Metrics
from keras.metrics import MeanIoU

In [3]:
# Get Data sets from the InBreast2012 Data-set on kaggle
DCM_PATH = '/kaggle/input/inbreast2012/INbreast Release 1.0/AllDICOMs/'
XML_PATH = '/kaggle/input/inbreast2012/INbreast Release 1.0/AllXML/'

In [ ]:
# List of patient IDs for the INBREAST dataset
MASS_PATIENT_ID = ['53586896', '22580192', '22614236', '22580098', '24055445', '30011674', '20586934', '22670465', '24055502', '22670673', '20587612', '22614568', '20587902', '22614522', '50995789', '24055464', '20588216', '51049053', '53582656', '20588562', '27829188', '22614431', '22580341', '22613822', '24065584', '50997515', '51049107', '22580367', '22580244', '50996352', '22670147', '22580732', '50999008', '24065707', '22614127', '20588334', '20588536', '24065530', '22670324', '20586908', '30011507', '27829134', '53581406', '50998981', '20586986', '22678787', '50997461', '53580804', '22579730', '22670094', '53580858', '53586869', '50995762', '24065251', '20587810', '53581460', '22670855', '22580706', '30011553', '22670809', '22580419', '24055355', '53587014', '50994408', '22614379', '22670278', '24065289', '22614074', '24055274', '22670511', '50994354', '20587928', '22580393', '22580654', '20588046', '50994273', '20587758', '24065761', '22427751', '20587664', '50999432', '22580680', '22580038', '53587663', '20588308', '20588680', '30011727', '22678833', '22427705', '22614266', '22613650', '50999459', '24055483', '22678694', '20587994', '22678646', '53582683', '20586960', '51048765', '22670620', '22613770', '22427840', '20588190', '53586960', '50996406', '22613702', '51048738']

seed = 40  # Seed to generate a different dataset

def load_inbreast_mask(mask_path, imshape=(4084, 3328)):
    """
    This function loads an OsiriX XML region as a binary numpy array for the INBREAST dataset.

    Args:
    @mask_path : Path to the XML file
    @imshape : The shape of the image as an array e.g. [4084, 3328]

    Returns:
    numpy array where each mass has a different number ID.
    """

    def load_point(point_string):
        # Helper function to extract x, y coordinates from a string
        x, y = tuple([float(num) for num in point_string.strip('()').split(',')])
        return y, x

    i = 0  # Counter for the number of masses detected
    mask = np.zeros(imshape)  # Initialize an array for the mask

    with open(mask_path, 'rb') as mask_file:
        plist_dict = plistlib.load(mask_file, fmt=plistlib.FMT_XML)['Images'][0]
        numRois = plist_dict['NumberOfROIs']  # Number of ROIs in the XML file
        rois = plist_dict['ROIs']  # Extract all ROIs
        assert len(rois) == numRois  # Ensure consistency in ROI count

        for roi in rois:
            numPoints = roi['NumberOfPoints']  # Number of points in the ROI
            if roi['Name'] == 'Mass':  # Check if the ROI is a 'Mass'
                i += 1  # Increment the mass counter
                points = roi['Point_px']  # Extract points of the mass
                assert numPoints == len(points)  # Ensure consistency in point count

                # Load and map points to the mask
                points = [load_point(point) for point in points]
                if len(points) <= 2:
                    for point in points:
                        mask[int(point[0]), int(point[1])] = i  # Mark points on the mask
                else:
                    x, y = zip(*points)
                    x, y = np.array(x), np.array(y)
                    poly_x, poly_y = polygon(x, y, shape=imshape)  # Create a polygon from points
                    mask[poly_x, poly_y] = i  # Fill the polygon on the mask with the mass ID

    return mask  # Return the generated mask

In [10]:
def load_dicom(dicom_path):
    # Load DICOM file
    dicom = pydicom.dcmread(dicom_path)

    # Extract pixel data from DICOM file
    pixel_data = dicom.pixel_array
    return pixel_data

def histogram_equalization_png(image_path):
    # Read the image using OpenCV
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Apply histogram equalization
    img_equalized = cv2.equalizeHist(img)
    return img_equalized

def histogram_equalization(img):
    m = int(np.max(img))
    hist = np.histogram(img, bins=m+1, range=(0, m+1))[0]
    # bước 1: tính pdf
    hist = hist/img.size
    # bước 2: tính cdf
    cdf = np.cumsum(hist)
    # bước 3: lập bảng thay thế
    s_k = (255 * cdf)
    # ảnh mới
    img_new = np.array([s_k[i] for i in img.ravel()]).reshape(img.shape)
    return img_new


In [ ]:
# Folder paths
DCM_PATH = '/kaggle/input/inbreast2012/INbreast Release 1.0/AllDICOMs/'
Y_TRAIN_PATH = '/kaggle/input/inbreast2012/INbreast Release 1.0/AllXML/'

# List all X_train files
X_train_files = os.listdir(DCM_PATH)

# Initialize lists for X_train and Y_train data
X_train_data = []
Y_train_data = []

# Define a regular expression pattern to match digits before underscore
pattern = re.compile(r'^(\d+)_')

# Iterate through X_train files
for x_file in X_train_files:
    match = pattern.match(x_file)
    if match:
        file_id = match.group(1)

        # Find the corresponding Y_train file in the Y_TRAIN_PATH folder
        y_file_candidates = [y_file for y_file in os.listdir(Y_TRAIN_PATH) if y_file.startswith(file_id)]

        if y_file_candidates:
            # Assuming you want to use the first matching Y_train file
            y_file = y_file_candidates[0]
            y_file_path = os.path.join(Y_TRAIN_PATH, y_file)

            # Load X_train and Y_train data (adjust this part based on your data format)
            x_data = load_dicom(os.path.join(DCM_PATH, x_file))
#             print(x_data.shape)
            y_data =load_inbreast_mask( y_file_path,  imshape=(x_data.shape[0], x_data.shape[1]))
            if y_data.max()<1: # if no lesion
                continue #skip
#             print(y_data.shape)
            # resize the data
            x_data_resized = cv2.resize(x_data, (256, 256))
            y_data_resized = cv2.resize(y_data, (256, 256))

            # Append resized data to respective lists
            X_train_data.append(x_data_resized)
            Y_train_data.append(y_data_resized)
        else:
            print(f"No corresponding Y_train file found for X_train file '{x_file}' with ID '{file_id}'.")
            
X_train_data = np.array(X_train_data)
Y_train_data = np.array(Y_train_data)


In [ ]:
# Loop through the first 20 pairs of training data (images and corresponding masks)
for i, (x_data, y_data) in enumerate(zip(X_train_data[:20], Y_train_data[:20])):
    
    # Create a figure to display the current pair of images
    plt.figure(figsize=(10, 5))
    
    # Plot the original image (x_data) on the left-hand side
    plt.subplot(1, 2, 1)
    plt.imshow(x_data, cmap='gray')  # Display the grayscale image (assuming x_data is grayscale)
    plt.title(f"Example {i + 1} - X_train")  # Set the title for the original image
    
    # Plot the corresponding mask (y_data) on the right-hand side
    plt.subplot(1, 2, 2)
    plt.imshow(y_data, cmap='gray')  # Display the mask (adjust the colormap based on mask data)
    plt.title(f"Example {i + 1} - Y_train (Mask)")  # Set the title for the mask
    
    plt.show()  # Show the plotted images

In [ ]:
from keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPooling2D, Conv2DTranspose, concatenate
from keras.models import Model

# Define the U-Net architecture
def unet(input_size=(256, 256, 1)):
    inputs = Input(input_size)
    
    # Contracting path (Encoder)
    conv1 = Conv2D(64, (3, 3), padding='same')(inputs)
    bn1 = BatchNormalization(axis=3)(conv1)
    bn1 = Activation('relu')(bn1)
    conv1 = Conv2D(64, (3, 3), padding='same')(bn1)
    bn1 = BatchNormalization(axis=3)(conv1)
    bn1 = Activation('relu')(bn1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(bn1)

    # Contracting path (Encoder)
    conv2 = Conv2D(128, (3, 3), padding='same')(pool1)
    bn2 = BatchNormalization(axis=3)(conv2)
    bn2 = Activation('relu')(bn2)
    conv2 = Conv2D(128, (3, 3), padding='same')(bn2)
    bn2 = BatchNormalization(axis=3)(conv2)
    bn2 = Activation('relu')(bn2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(bn2)

    # Contracting path (Encoder)
    conv3 = Conv2D(256, (3, 3), padding='same')(pool2)
    bn3 = BatchNormalization(axis=3)(conv3)
    bn3 = Activation('relu')(bn3)
    conv3 = Conv2D(256, (3, 3), padding='same')(bn3)
    bn3 = BatchNormalization(axis=3)(conv3)
    bn3 = Activation('relu')(bn3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(bn3)

    # Contracting path (Encoder)
    conv4 = Conv2D(512, (3, 3), padding='same')(pool3)
    bn4 = BatchNormalization(axis=3)(conv4)
    bn4 = Activation('relu')(bn4)
    conv4 = Conv2D(512, (3, 3), padding='same')(bn4)
    bn4 = BatchNormalization(axis=3)(conv4)
    bn4 = Activation('relu')(bn4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(bn4)

    # Contracting path (Encoder)
    conv5 = Conv2D(1024, (3, 3), padding='same')(pool4)
    bn5 = BatchNormalization(axis=3)(conv5)
    bn5 = Activation('relu')(bn5)
    conv5 = Conv2D(1024, (3, 3), padding='same')(bn5)
    bn5 = BatchNormalization(axis=3)(conv5)
    bn5 = Activation('relu')(bn5)

    # Expansive path (Decoder)
    up6 = concatenate([Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(bn5), conv4], axis=3)
    conv6 = Conv2D(512, (3, 3), padding='same')(up6)
    bn6 = BatchNormalization(axis=3)(conv6)
    bn6 = Activation('relu')(bn6)
    conv6 = Conv2D(512, (3, 3), padding='same')(bn6)
    bn6 = BatchNormalization(axis=3)(conv6)
    bn6 = Activation('relu')(bn6)

    # Expansive path (Decoder)
    up7 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(bn6), conv3], axis=3)
    conv7 = Conv2D(256, (3, 3), padding='same')(up7)
    bn7 = BatchNormalization(axis=3)(conv7)
    bn7 = Activation('relu')(bn7)
    conv7 = Conv2D(256, (3, 3), padding='same')(bn7)
    bn7 = BatchNormalization(axis=3)(conv7)
    bn7 = Activation('relu')(bn7)

    # Expansive path (Decoder)
    up8 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(bn7), conv2], axis=3)
    conv8 = Conv2D(128, (3, 3), padding='same')(up8)
    bn8 = BatchNormalization(axis=3)(conv8)
    bn8 = Activation('relu')(bn8)
    conv8 = Conv2D(128, (3, 3), padding='same')(bn8)
    bn8 = BatchNormalization(axis=3)(conv8)
    bn8 = Activation('relu')(bn8)

    # Expansive path (Decoder)
    up9 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(bn8), conv1], axis=3)
    conv9 = Conv2D(64, (3, 3), padding='same')(up9)
    bn9 = BatchNormalization(axis=3)(conv9)
    bn9 = Activation('relu')(bn9)
    conv9 = Conv2D(64, (3, 3), padding='same')(bn9)
    bn9 = BatchNormalization(axis=3)(conv9)
    bn9 = Activation('relu')(bn9)

    # Output layer
    conv10 = Conv2D(1, (1, 1), activation='sigmoid')(bn9)

    # Create the U-Net model
    return Model(inputs=[inputs], outputs=[conv10])


In [ ]:
# Create a U-Net model using the defined architecture
model = unet()

# Display a summary of the model architecture
model.summary()

In [ ]:
# Number of epochs determines how many times the entire dataset is passed through the neural network during training
EPOCHS = 100

# Batch size defines the number of training examples processed in one iteration during training
BATCH_SIZE = 16

# Learning rate sets the rate at which the model's parameters are updated during training
learning_rate = 1e-3

# Smooth parameter used in loss functions to prevent extreme values, often added to predicted and target values
smooth = 0.001

In [ ]:
def dice_coef(y_true, y_pred, smooth=0.001):
    # Calculate intersection by summing the element-wise product of true and predicted masks
    intersection = K.sum(y_true * y_pred, axis=[1, 2, 3])

    # Calculate union by summing the elements in true and predicted masks separately
    union = K.sum(y_true, axis=[1, 2, 3]) + K.sum(y_pred, axis=[1, 2, 3])

    # Compute Dice coefficient: (2 * intersection + smooth) / (union + smooth)
    dice = (2. * intersection + smooth) / (union + smooth)

    # Return Dice coefficient
    return dice  # We usually minimize loss, so maximize (1 - dice)

def dice_coef_loss(y_true, y_pred):
    # Define loss as 1 minus the Dice coefficient to be used as the loss function
    return 1 - dice_coef(y_true, y_pred)

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf

# Expand dimensions of training data to match the input shape of the model
X_train_data = np.expand_dims(X_train_data, axis=-1)
Y_train_data = np.expand_dims(Y_train_data, axis=-1)

# Split the data into training and validation sets (80% training, 20% validation)
X_train, X_val, Y_train, Y_val = train_test_split(
    X_train_data, Y_train_data, test_size=0.2, random_state=42
)

# Create a U-Net model with the specified input size
model = unet(input_size=(256, 256, 1))

# Calculate the decay rate for the learning rate schedule
decay_rate = learning_rate / EPOCHS

# Define the optimizer (Adam) with a specified initial learning rate
opt = tf.keras.optimizers.Adam(learning_rate=0.01)

# Compile the model with the optimizer, loss function, and metrics for evaluation
model.compile(optimizer=opt, loss=dice_coef_loss, metrics=["binary_accuracy", dice_coef])

# Train the model using the training data and validate on the validation set
history = model.fit(
    X_train, (Y_train > 0).astype(float),
    validation_data=(X_val, (Y_val > 0).astype(float)),
    steps_per_epoch=len(X_train) // 16,  # Number of steps per epoch (batch size)
    epochs=EPOCHS,  # Number of training epochs
    batch_size=16  # Batch size for training
)

# Evaluate the model on the validation set and output metrics
val_metrics = model.evaluate(X_val, (Y_val > 0).astype(float), batch_size=16)
print(f"Validation Metrics: {dict(zip(model.metrics_names, val_metrics))}")

In [ ]:
# Save the trained U-Net model to a file path specified
model.save('/kaggle/working/Unet_segmenter.keras')

In [ ]:
# Save only the weights of the trained U-Net model to a file path specified
model.save_weights('/kaggle/working/Unet_segmenter_weights.h5')